In [ ]:
# Импортируем библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import cross_val_score
import numpy as np

# Импортируем csv файл с данными
data = pd.read_csv('/content/encoded-data.csv')

# Преобразование времени подъема в минуты
def time_to_minutes(time_str):
    hours, minutes = map(int, time_str.split(':'))
    return hours * 60 + minutes # Возвращаем время в минутах

data['wake_time'] = data['wake_time'].apply(time_to_minutes) # Применяем функцию и записываем значения в столбец wake_time

# Посмотрим на структуру данных
print(data.head())

# Преобразование категориальных переменных в числовые
label_encoders = {}
categorical_columns = ['gender', 'smoking', 'eye_color', 'well_slept',
                       'chronotype', 'near_coffee_shop', 'gourmet',
                       'office_worker', 'homebody', 'chronic_diseases',
                       'handedness', 'zodiac']

for column in categorical_columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Преобразование целевой переменной 'preferred_drink'
le_drink = LabelEncoder()
data['preferred_drink'] = le_drink.fit_transform(data['preferred_drink'])


# Подготовка признаков (X) и целевой переменной (y)
X = data.drop('preferred_drink', axis=1)  # Все, кроме preferred_drink
y = data['preferred_drink']  # Целевая переменная

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    gender preferred_drink  age  healthy_lifestyle smoking     eye_color  \
0  Женщина            Кофе   22                 65      Да  Серо-зеленый   
1  Мужчина             Чай   21                 85     Нет       Зеленый   
2  Мужчина            Кофе   22                 50      Да       Голубой   
3  Мужчина            Кофе   22                 80      Да         Карий   
4  Женщина            Кофе   23                 50     Нет       Голубой   

   stress_level well_slept chronotype  wake_time  sleep_on_average  \
0            55         Да  Жаворонок        480               8.0   
1            70         Да  Жаворонок        360               8.0   
2            70         Да       Сова        540               7.0   
3            60         Да       Сова        540               6.0   
4            90        Нет       Сова        840               6.0   

  near_coffee_shop gourmet office_worker homebody chronic_diseases handedness  \
0               Да      Да            Да 

Стандартизация преобразует все признаки к единому масштабу, чтобы каждый из них имел среднее значение 0 и стандартное отклонение 1.
Стандартизация данных необходима для корректной работы модели k-NN, чтобы избежать доминирования признаков с большими масштабами и сделать все признаки одинаково важными для вычисления расстояний между объектами.



**Метод кросс-валидации** — это техника оценки качества моделей машинного обучения, при которой данные многократно разделяются на обучающую и тестовую выборки для более надежной оценки модели. Это позволяет избежать переобучения или недообучения модели и предоставляет объективную оценку её производительности на независимых данных.

В нашей работе применяется **5-fold кросс-валидация**:
1. Данные X train разбиваются на 5 частей.
2. Модель обучается на 4 частях и тестируется на одной.
3. Этот процесс повторяется 5 раз, и в конце для каждого k вычисляется среднее значение точности.
4. Это позволяет выбрать лучшее значение гиперпараметра k для модели k-NN, основываясь на более надежной оценке производительности.

*Преимущества кросс-валидации:*
1. Использует все данные: Каждое наблюдение используется как для обучения, так и для тестирования, что делает оценку модели более точной.
2. Снижает вероятность переобучения: За счёт многократного тестирования на разных частях данных.
3. Лучший выбор гиперпараметров: Кросс-валидация часто используется для поиска оптимальных параметров модели (например, числа соседей в k-NN).

**Кросс-валидация** — это мощный метод для оценки качества модели, который даёт более надёжные результаты, чем однократное разделение на обучающую и тестовую выборки.

In [ ]:
# Стандартизация данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) # Здесь вычисляются среднее и стандартное отклонение для каждого признака на обучающей выборке, и затем все признаки стандартизируются.
X_test_scaled = scaler.transform(X_test) # Для тестовых данных применяется те же параметры стандартизации (среднее и стандартное отклонение), что были рассчитаны на обучающих данных. Это необходимо для того, чтобы сохранить консистентность преобразований.

# Выбор оптимального k с помощью кросс-валидации
k_values = range(1, 21)  # Тестируем значения k от 1 до 20
cross_val_scores = [] # Создаем пусто список для хранения результатов кросс-валидации для каждого значения k

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train_scaled, y_train, cv=5)
    cross_val_scores.append(scores.mean()) # Вычисляется среднее значение точности для каждого значения k (scores.mean()) и добавляется в список cross_val_scores.

# Найдем лучшее значение k
best_k = k_values[np.argmax(cross_val_scores)] # np.argmax(cross_val_scores) находит индекс максимального значения в списке cross_val_scores, то есть то значение k, для которого средняя точность была максимальной.
print(f'Лучшее значение k: {best_k}')

Лучшее значение k: 5


In [ ]:
# Обучение модели k-NN с выбранным значением k
knn = KNeighborsClassifier(n_neighbors=best_k) #Создаётся объект модели k-NN (k-ближайших соседей) из библиотеки scikit-learn, который указывает количество ближайших соседий (k=5)
knn.fit(X_train_scaled, y_train) # Модель обучается на тренировочных данных

# Предсказание
y_pred = knn.predict(X_test_scaled) # Выполняется предсказание для тестовых данных. Результатом является массив y_pred, содержащий предсказанные классы для каждого примера из тестовой выборки


**accuracy_score(y_test, y_pred)** вычисляет точность модели, то есть долю правильно предсказанных примеров из общего числа примеров.

y_test — это реальные метки классов для тестовой выборки.

y_pred — это предсказанные метки, полученные ранее.
Значение точности выводится в консоль.

**classification_report(y_test, y_pred)** выводит подробный отчёт о качестве модели, который включает:
1. Precision (точность): отношение истинных положительных предсказаний к общему числу положительных предсказаний.
2. Recall (полнота): отношение истинных положительных предсказаний к общему числу истинных положительных случаев.
3. F1-score: гармоническое среднее между точностью и полнотой.
4. Support: количество реальных примеров каждого класса в тестовой выборке.

In [ ]:
# Оценка модели
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

# Пример предсказания для новых данных
new_data = {
    'gender': 'Мужчина',
    'age': 30,
    'healthy_lifestyle': 80,
    'smoking': 'Нет',
    'eye_color': 'Карий',
    'stress_level': 40,
    'well_slept': 'Да',
    'chronotype': 'Жаворонок',
    'wake_time': '07:00',  # Это нужно будет изменить перед вводом
    'sleep_on_average': 7,
    'near_coffee_shop': 'Да',
    'gourmet': 'Нет',
    'office_worker': 'Да',
    'homebody': 'Нет',
    'chronic_diseases': 'Нет',
    'handedness': 'Правой',
    'zodiac': 'Овен'
}

# Преобразуем новые данные в формат для модели
new_data_df = pd.DataFrame([new_data])
new_data_df['wake_time'] = new_data_df['wake_time'].apply(time_to_minutes)

#Преобразуем категориальные данные
for column in categorical_columns:
    new_data_df[column] = label_encoders[column].transform(new_data_df[column])

# Стандартизируем новые данные
new_data_scaled = scaler.transform(new_data_df)

# Предсказание для новых данных
#Используем метод .predict() для предсказания класса на основе новых данных. Модель возвращает числовое значение, которое соответствует предсказанному классу напитка.
prediction = knn.predict(new_data_scaled)

# Обратное преобразование предсказания в исходную категорию
predicted_drink = le_drink.inverse_transform(prediction) #Получаем имя напитка "Чай" или "Кофе"

print(f'Предсказание для новых данных: {predicted_drink[0]}') #Выводим предсказанный напиток в удобочитаемом виде в консоль

Accuracy: 0.375
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.60      0.50      0.55         6

    accuracy                           0.38         8
   macro avg       0.30      0.25      0.27         8
weighted avg       0.45      0.38      0.41         8

Предсказание для новых данных: Кофе




```
# Пример предсказания новых данных: Чай
new_data = {
    'gender': 'Женщина',
    'age': 21,
    'healthy_lifestyle': 55,
    'smoking': 'Да',
    'eye_color': 'Зеленый',
    'stress_level': 40,
    'well_slept': 'Да',
    'chronotype': 'Жаворонок',
    'wake_time': '07:00', 
    'sleep_on_average': 7,
    'near_coffee_shop': 'Да',
    'gourmet': 'Нет',
    'office_worker': 'Да',
    'homebody': 'Нет',
    'chronic_diseases': 'Нет',
    'handedness': 'Правой',
    'zodiac': 'Овен'
}
```

